In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import gc
gc.enable()
gc.collect()

pd.options.display.max_columns = 100

%matplotlib inline

In [2]:
train = pd.read_csv('./training.csv')
test = pd.read_csv('./test.csv')
train.shape, test.shape

((95662, 16), (45019, 15))

In [3]:
#for col in ['TransactionId', 'BatchId', 'AccountId', 'SubscriptionId', 'CustomerId', 'ProviderId', 'ProductId', 
#            'ChannelId']:
#    train[col] = train[col].str.extract('(\d+)')
#    train[col] = train[col].astype('int')
    
#    test[col] = test[col].str.extract('(\d+)')
#    test[col] = test[col].astype('int')

In [4]:
train['TransactionStartTime'] = train['TransactionStartTime'].str.replace('T', ' ')
train['TransactionStartTime'] = train['TransactionStartTime'].str.replace('Z', '')

test['TransactionStartTime'] = test['TransactionStartTime'].str.replace('T', ' ')
test['TransactionStartTime'] = test['TransactionStartTime'].str.replace('Z', '')

In [5]:
#train['Date'] = train['TransactionStartTime'].str.extract('(\d+-\d+-\d+)T')
#train['Time'] = train['TransactionStartTime'].str.extract('(\d+:\d+:\d+)Z')

In [6]:
train['TransactionStartTime'] = pd.to_datetime(train['TransactionStartTime'], infer_datetime_format=True) 
test['TransactionStartTime'] = pd.to_datetime(test['TransactionStartTime'], infer_datetime_format=True) 

In [7]:
train['Year'] = train['TransactionStartTime'].dt.year
train['Month'] = train['TransactionStartTime'].dt.month
train['Day'] = train['TransactionStartTime'].dt.day
train['Hour'] = train['TransactionStartTime'].dt.hour
train['Minute'] = train['TransactionStartTime'].dt.minute
train['Seconds'] = train['TransactionStartTime'].dt.second
train['week'] = train['TransactionStartTime'].dt.week
train['weekday'] = train['TransactionStartTime'].dt.weekday

test['Year'] = test['TransactionStartTime'].dt.year
test['Month'] = test['TransactionStartTime'].dt.month
test['Day'] = test['TransactionStartTime'].dt.day
test['Hour'] = test['TransactionStartTime'].dt.hour
test['Minute'] = test['TransactionStartTime'].dt.minute
test['Seconds'] = test['TransactionStartTime'].dt.second
test['week'] = test['TransactionStartTime'].dt.week
test['weekday'] = test['TransactionStartTime'].dt.weekday

In [8]:
train.loc[train['Amount'] > 0, 'Debit'] = 1
train.loc[train['Amount'] < 0, 'Debit'] = 0

test.loc[test['Amount'] > 0, 'Debit'] = 1
test.loc[test['Amount'] < 0, 'Debit'] = 0

train['Debit'] = train['Debit'].astype('int')
test['Debit'] = test['Debit'].astype('int')

In [9]:
all_df = pd.concat([train, test]).reset_index(drop=True)
all_df = all_df.sort_values(by='TransactionStartTime').reset_index(drop=True)
all_df.shape

/home/raj/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


(140681, 25)

In [10]:
additional_cols = ['CustomerId', 'AccountId', 'TransactionStartTime', 'CountryCode']
new_df = all_df[['TransactionId']+ additional_cols]

In [11]:
for col in ['Value', 'Day', 'Seconds', 'Amount']:
    gdf = all_df.groupby('CustomerId')[col].mean().reset_index()
    gdf.columns = ['CustomerId', 'Customer_'+col+'_mean']
    new_df = pd.merge(new_df, gdf, on='CustomerId', how='left')
    
for col in ['Value', 'Day', 'Seconds', 'Amount', 'Debit']:
    gdf = all_df.groupby('CustomerId')[col].sum().reset_index()
    gdf.columns = ['CustomerId', 'Customer_'+col+'_sum']
    new_df = pd.merge(new_df, gdf, on='CustomerId', how='left')
    
#for col in ['Value', 'Seconds', 'week', 'Amount']:
#    gdf = all_df.groupby('CustomerId')[col].cumsum().reset_index()
#    gdf.columns = ['CustomerId', 'Customer_'+col+'_cumsum']
#    new_df = pd.merge(new_df, gdf, on='CustomerId', how='left')

for col in ['AccountId', 'BatchId', 'SubscriptionId', 'ChannelId', 'ProductId', 'ProviderId']:
    gdf = all_df.groupby('CustomerId')[col].nunique().reset_index()
    gdf.columns = ['CustomerId', 'Customer_'+col+'_nunique']
    new_df = pd.merge(new_df, gdf, on='CustomerId', how='left')

In [12]:
new_df['Customer_cumcount_seconds'] = all_df.groupby('CustomerId')['Seconds'].cumcount().values
new_df['Customer_cumsum_day'] = all_df.groupby('CustomerId')['Day'].cumsum().values
new_df['Customer_cumsum_month'] = all_df.groupby('CustomerId')['Month'].cumsum().values

#all_df['prev_trans_date'] = all_df.groupby('CustomerId')['Seconds'].shift(1)
#new_df['time_gap_prev'] = all_df['Seconds'] - all_df['prev_trans_date']

#all_df['next_trans_date'] = all_df.groupby('CustomerId')['Seconds'].shift(-1)
#new_df['time_gap_next'] = all_df['Seconds'] - all_df['next_trans_date']

#all_df['prev_trans_date2'] = all_df.groupby('CustomerId')['Seconds'].shift(2)
#new_df['time_gap_prev2'] = all_df['Seconds'] - all_df['prev_trans_date2']

#all_df['next_trans_date2'] = all_df.groupby('CustomerId')['Seconds'].shift(-2)
#new_df['time_gap_next2'] = all_df['Seconds'] - all_df['next_trans_date2']

In [13]:
gdf = pd.pivot_table(all_df, index="CustomerId", columns="Year", values="TransactionId", aggfunc="count", 
                     fill_value=0).reset_index()
new_df = pd.merge(new_df, gdf, on="CustomerId", how="left")

gdf = pd.pivot_table(all_df, index="CustomerId", columns="Month", values="TransactionId", aggfunc="count", 
                     fill_value=0).reset_index()
new_df = pd.merge(new_df, gdf, on="CustomerId", how="left")

gdf = pd.pivot_table(all_df, index="CustomerId", columns="week", values="TransactionId", aggfunc="count", 
                     fill_value=0).reset_index()
new_df = pd.merge(new_df, gdf, on="CustomerId", how="left")

gdf = pd.pivot_table(all_df, index="CustomerId", columns="weekday", values="TransactionId", aggfunc="count", 
                     fill_value=0).reset_index()
new_df = pd.merge(new_df, gdf, on="CustomerId", how="left")

In [14]:
gdf = pd.pivot_table(all_df, index="CustomerId", columns="ChannelId", values="TransactionId", aggfunc="count", 
                     fill_value=0).reset_index()
new_df = pd.merge(new_df, gdf, on="CustomerId", how="left")

gdf = pd.pivot_table(all_df, index="CustomerId", columns="ProviderId", values="TransactionId", aggfunc="count", 
                     fill_value=0).reset_index()
new_df = pd.merge(new_df, gdf, on="CustomerId", how="left")

gdf = pd.pivot_table(all_df, index="CustomerId", columns="ProductId", values="TransactionId", aggfunc="count", 
                     fill_value=0).reset_index()
new_df = pd.merge(new_df, gdf, on="CustomerId", how="left")

In [15]:
new_df = new_df.drop(additional_cols, axis=1)
new_df.head()

,TransactionId,Customer_Value_mean,Customer_Day_mean,Customer_Seconds_mean,Customer_Amount_mean,Customer_Value_sum,Customer_Day_sum,Customer_Seconds_sum,Customer_Amount_sum,Customer_Debit_sum,Customer_AccountId_nunique,Customer_BatchId_nunique,Customer_SubscriptionId_nunique,Customer_ChannelId_nunique,Customer_ProductId_nunique,Customer_ProviderId_nunique,Customer_cumcount_seconds,Customer_cumsum_day,Customer_cumsum_month,2018,2019,1_x,2_x,3_x,11_x,12,1_y,2_y,3_y,4_x,5_x,6_x,7,8,9,10,11_y,46,47,48,49,50,51,52,0,1,2,3,4_y,5_y,6_y,ChannelId_1,ChannelId_2,ChannelId_3,ChannelId_4,ChannelId_5,ProviderId_1,ProviderId_2,ProviderId_3,ProviderId_4,ProviderId_5,ProviderId_6,ProductId_1,ProductId_10,ProductId_11,ProductId_12,ProductId_13,ProductId_14,ProductId_15,ProductId_16,ProductId_17,ProductId_18,ProductId_19,ProductId_2,ProductId_20,ProductId_21,ProductId_22,ProductId_23,ProductId_24,ProductId_25,ProductId_26,ProductId_27,ProductId_3,ProductId_4,ProductId_5,ProductId_6,ProductId_7,ProductId_8,ProductId_9
0,TransactionId_76871,1643.013158,14.802632,30.223684,832.468750,249738,2250,4594,126535.25,82,4,151,3,3,7,5,0,15,11,89,63,10,46,7,47,42,3,1,0,5,1,19,7,11,11,2,3,12,27,11,27,5,4,3,37,23,21,15,27,17,12,0,41,88,0,23,1,0,6,70,29,46,0,42,3,0,0,26,5,0,0,0,0,0,0,0,0,0,0,0,0,0,26,3,0,47,0,0,0
1,TransactionId_73770,1643.013158,14.802632,30.223684,832.468750,249738,2250,4594,126535.25,82,4,151,3,3,7,5,1,30,22,89,63,10,46,7,47,42,3,1,0,5,1,19,7,11,11,2,3,12,27,11,27,5,4,3,37,23,21,15,27,17,12,0,41,88,0,23,1,0,6,70,29,46,0,42,3,0,0,26,5,0,0,0,0,0,0,0,0,0,0,0,0,0,26,3,0,47,0,0,0
2,TransactionId_26203,500.000000,19.500000,23.000000,500.000000,1000,39,46,1000.00,2,1,2,1,1,2,1,0,15,11,2,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,2,0,0,0,0,0,0,0,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,TransactionId_380,6502.163636,14.290909,34.127273,4830.567273,357619,786,1877,265681.20,36,3,55,3,3,10,4,0,15,11,23,32,10,5,17,5,18,15,3,0,0,2,3,0,0,0,12,5,4,1,0,1,0,2,7,15,6,2,10,17,2,3,0,11,42,0,2,9,0,0,19,18,9,0,5,2,0,0,1,3,0,0,0,2,0,0,8,1,0,0,0,0,0,14,2,0,17,0,0,0
4,TransactionId_28195,6502.163636,14.290909,34.127273,4830.567273,357619,786,1877,265681.20,36,3,55,3,3,10,4,1,30,22,23,32,10,5,17,5,18,15,3,0,0,2,3,0,0,0,12,5,4,1,0,1,0,2,7,15,6,2,10,17,2,3,0,11,42,0,2,9,0,0,19,18,9,0,5,2,0,0,1,3,0,0,0,2,0,0,8,1,0,0,0,0,0,14,2,0,17,0,0,0


In [16]:
new_df.to_csv('Feat_v4.csv', index=False)